In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/24 19:57:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/24 19:57:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
empleados = spark.read.parquet('./data/empleados')

departamentos = spark.read.parquet('./data/departamentos')

In [3]:
from pyspark.sql.functions import col, broadcast

In [5]:
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [6]:
empleados.join(broadcast(departamentos), col('num_dpto') == col('id')).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#1L], [id#4L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#1L)
   :  +- FileScan parquet [nombre#0,num_dpto#1L] Batched: true, DataFilters: [isnotnull(num_dpto#1L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/rando/GitHub/pyspark_de/data/empleados], PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=193]
      +- Filter isnotnull(id#4L)
         +- FileScan parquet [id#4L,nombre_dpto#5] Batched: true, DataFilters: [isnotnull(id#4L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/rando/GitHub/pyspark_de/data/departamentos], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:string>


